In [1]:
!pip install langchain langchain-community chromadb fastapi uvicorn gTTS transformers accelerate llama-cpp-python
# If you want GPU support for llama-cpp-python
# !pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121

In [8]:
import os
from huggingface_hub import hf_hub_download

# --- LLM Model Configuration ---
# This is the corrected repo ID and filename for OpenHathi-7B-Hi-v0.1-Base GGUF
repo_id = "sarvamai/OpenHathi-7B-Hi-v0.1-Base"
filename = "OpenHathi-7B-Hi-v0.1-Base-q4_0.gguf" # This is a specific GGUF file within that repo

# Create a directory for models
model_dir = "./models"
os.makedirs(model_dir, exist_ok=True) # Ensure models directory exists

LLM_MODEL_PATH = os.path.join(model_dir, filename)

if not os.path.exists(LLM_MODEL_PATH):
    print(f"Downloading {filename} from {repo_id}...")
    try:
        hf_hub_download(repo_id=repo_id, filename=filename, local_dir=model_dir, local_dir_use_symlinks=False)
        print("Download complete.")
    except Exception as e:
        print(f"Error downloading model: {e}")
        print("Please ensure the repo_id and filename are correct and you have access if it's a gated model.")
        print("If it's a gated model, make sure you've accepted the terms on its Hugging Face page and set your HF_TOKEN in Colab secrets.")
else:
    print(f"Model already exists at {LLM_MODEL_PATH}")

# --- Directory Setup for Static Files and ChromaDB ---
# Ensure the 'static' directory exists before FastAPI tries to mount it
os.makedirs("static", exist_ok=True)
print("Created 'static' directory for audio files.")

# Path for ChromaDB persistence
# You can change this to a Google Drive path if you want persistence across sessions:
# from google.colab import drive
# drive.mount('/content/drive')
# CHROMA_DB_PATH = "/content/drive/MyDrive/my_msme_bot_data/chroma_db"
CHROMA_DB_PATH = "./chroma_db"
os.makedirs(CHROMA_DB_PATH, exist_ok=True) # Ensure ChromaDB directory exists
print(f"ChromaDB persistence directory set to: {CHROMA_DB_PATH}")

Model already exists at ./models/OpenHathi-7B-Hi-v0.1-Base-q4_0.gguf
Created 'static' directory for audio files.
ChromaDB persistence directory set to: ./chroma_db


In [9]:
from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse, FileResponse
from typing import List, Dict
import os

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# TTS imports
from gtts import gTTS

# --- FastAPI App Definition ---
app = FastAPI()

# --- Initialize LangChain Components (Singleton Pattern) ---
# Global variables to store initialized components
qa_chain = None
embeddings_model = None
vector_store = None
llm_model = None

def initialize_langchain():
    global qa_chain, embeddings_model, vector_store, llm_model

    if qa_chain is not None:
        print("LangChain components already initialized.")
        return # Already initialized

    print("Initializing LangChain components for the first time...")

    # 1. Embeddings Model
    print("Loading Embeddings Model...")
    embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    print("Embeddings Model Loaded.")

    # 2. Vector Store (ChromaDB)
    # The directory creation is handled in Cell 2, ensuring it exists before this
    print(f"Loading/Creating ChromaDB at {CHROMA_DB_PATH}...")
    vector_store = Chroma(
        embedding_function=embeddings_model,
        persist_directory=CHROMA_DB_PATH
    )
    print(f"ChromaDB initialized. Documents currently in store: {vector_store._collection.count()}")

    # Add some initial documents if the collection is empty (for demo)
    if vector_store._collection.count() == 0:
        print("Adding initial Hindi documents to ChromaDB...")
        sample_docs_hindi = [
            "सूक्ष्म, लघु और मध्यम उद्यम (MSME) भारत की अर्थव्यवस्था की रीढ़ हैं। सरकार MSME को बढ़ावा देने के लिए कई योजनाएं चला रही है, जैसे मुद्रा योजना और स्टैंड-अप इंडिया योजना।",
            "मुद्रा योजना छोटे व्यवसायों को ₹10 लाख तक का ऋण प्रदान करती है ताकि वे अपने व्यापार का विस्तार कर सकें या नया व्यवसाय शुरू कर सकें।",
            "पंजीकरण के लिए, आपको उद्योग आधार पोर्टल पर ऑनलाइन आवेदन करना होगा। यह प्रक्रिया सरल और मुफ्त है।",
            "अपने व्यवसाय के लिए मार्केटिंग अभियान बनाने के लिए, आप लक्षित दर्शकों, संदेश और बजट को परिभाषित कर सकते हैं। सोशल मीडिया मार्केटिंग छोटे व्यवसायों के लिए बहुत प्रभावी हो सकती है।",
            "स्टैंड-अप इंडिया योजना महिला उद्यमियों और अनुसूचित जाति/जनजाति के उद्यमियों को ₹10 लाख से ₹1 करोड़ तक का ऋण प्रदान करती है ताकि वे ग्रीनफ़ील्ड उद्यम स्थापित कर सकें।",
            "एक प्रभावी विज्ञापन बनाने के लिए, आपको अपने उत्पाद या सेवा के मुख्य लाभों पर ध्यान केंद्रित करना चाहिए और एक स्पष्ट कॉल-टू-एक्शन शामिल करना चाहिए।"
        ]
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        docs = text_splitter.create_documents(sample_docs_hindi)
        vector_store.add_documents(docs)
        print("Initial documents added.")
        vector_store.persist() # Important: Persist the database after adding documents
        print("ChromaDB persisted with initial documents.")
    else:
        print("ChromaDB already contains documents, skipping initial addition.")

    # 3. LLM (Large Language Model)
    # LLM_MODEL_PATH is defined in Cell 2
    print(f"Attempting to load LLM from: {LLM_MODEL_PATH}")
    if not os.path.exists(LLM_MODEL_PATH):
         print(f"WARNING: LLM model not found at {LLM_MODEL_PATH}. Using a dummy LLM for basic responses.")
         class DummyLLM:
             def invoke(self, prompt):
                 print(f"Dummy LLM received prompt: {prompt[:100]}...")
                 if "मुद्रा योजना" in prompt:
                     return "मुद्रा योजना छोटे व्यवसायों को ₹10 लाख तक का ऋण देती है ताकि वे अपना व्यवसाय शुरू या विस्तार कर सकें।"
                 elif "पंजीकरण" in prompt:
                     return "व्यवसाय पंजीकरण के लिए, आपको उद्योग आधार पोर्टल पर ऑनलाइन आवेदन करना होगा। यह प्रक्रिया सरल और मुफ्त है।"
                 elif "मार्केटिंग" in prompt:
                     return "मार्केटिंग के लिए सोशल मीडिया, लक्षित विज्ञापन और ग्राहक संबंध बनाना अच्छे तरीके हैं।"
                 elif "अभियान" in prompt and "उत्पाद" in prompt:
                     # Dummy response for campaign generation
                     if "छूट" in prompt and "वैधता" in prompt:
                         return f"आपके '{prompt.split('उत्पाद/सेवा: ')[1].splitlines()[0]}' उत्पाद पर '{prompt.split('छूट: ')[1].splitlines()[0]}' की शानदार छूट! यह ऑफर केवल '{prompt.split('वैधता: ')[1].splitlines()[0]}' तक वैध है। आज ही लाभ उठाएं!"
                     return f"आपके '{prompt.split('उत्पाद/सेवा: ')[1].splitlines()[0]}' उत्पाद के लिए एक शानदार अभियान!"
                 return "मुझे इस प्रश्न का उत्तर नहीं मिल रहा है। कृपया अधिक जानकारी के लिए पूछें।"
         llm_model = DummyLLM()
    else:
        try:
            llm_model = LlamaCpp(
                model_path=LLM_MODEL_PATH,
                temperature=0.7,
                max_tokens=500,
                n_gpu_layers=0, # Set to 0 for CPU inference. Set > 0 (e.g., -1 for all) if you installed with CUDA support and have a GPU.
                n_batch=512,
                f16_kv=True,
                verbose=False,
                n_ctx=2048 # Context window size, match model capabilities
            )
            print("LlamaCpp LLM loaded successfully.")
        except Exception as e:
            print(f"Error loading LlamaCpp model from {LLM_MODEL_PATH}: {e}")
            print("Falling back to Dummy LLM. Please check your LlamaCpp installation and model file.")
            class DummyLLM:
                 def invoke(self, prompt):
                     print(f"Dummy LLM received prompt: {prompt[:100]}...")
                     if "मुद्रा योजना" in prompt:
                         return "मुद्रा योजना छोटे व्यवसायों को ₹10 लाख तक का ऋण देती है ताकि वे अपना व्यवसाय शुरू या विस्तार कर सकें।"
                     elif "पंजीकरण" in prompt:
                         return "व्यवसाय पंजीकरण के लिए, आपको उद्योग आधार पोर्टल पर ऑनलाइन आवेदन करना होगा। यह प्रक्रिया सरल और मुफ्त है।"
                     elif "मार्केटिंग" in prompt:
                         return "मार्केटिंग के लिए सोशल मीडिया, लक्षित विज्ञापन और ग्राहक संबंध बनाना अच्छे तरीके हैं।"
                     elif "अभियान" in prompt and "उत्पाद" in prompt:
                         if "छूट" in prompt and "वैधता" in prompt:
                             return f"आपके '{prompt.split('उत्पाद/सेवा: ')[1].splitlines()[0]}' उत्पाद पर '{prompt.split('छूट: ')[1].splitlines()[0]}' की शानदार छूट! यह ऑफर केवल '{prompt.split('वैधता: ')[1].splitlines()[0]}' तक वैध है। आज ही लाभ उठाएं!"
                         return f"आपके '{prompt.split('उत्पाद/सेवा: ')[1].splitlines()[0]}' उत्पाद के लिए एक शानदार अभियान!"
                     return "मुझे इस प्रश्न का उत्तर नहीं मिल रहा है। कृपया अधिक जानकारी के लिए पूछें।"
            llm_model = DummyLLM()


    # 4. RAG Chain
    print("Setting up RetrievalQA Chain...")
    prompt_template_hindi = """आप एक हिंदी भाषी सहायक हैं जो भारतीय सूक्ष्म, लघु और मध्यम उद्यमों (MSME) की सहायता के लिए डिज़ाइन किए गए हैं।
    दिए गए संदर्भ से ही उत्तर दें। यदि उत्तर संदर्भ में नहीं है, तो बस कहें कि "मुझे इस प्रश्न का उत्तर नहीं मिल रहा है।"
    किसी भी तरह से कोई भी गलत जानकारी न दें।

    संदर्भ:
    {context}

    प्रश्न: {question}
    उत्तर:"""
    PROMPT = PromptTemplate(template=prompt_template_hindi, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm_model,
        chain_type="stuff", # Simple stuffing of context into prompt
        retriever=vector_store.as_retriever(),
        return_source_documents=True, # Optional: to show what documents were used
        chain_type_kwargs={"prompt": PROMPT}
    )
    print("LangChain components initialized successfully.")

# --- API Endpoints ---
# Using lifespan context manager for FastAPI startup/shutdown events
# This replaces the deprecated @app.on_event("startup")
from contextlib import asynccontextmanager

@asynccontextmanager
async def lifespan(app: FastAPI):
    # This code runs on startup
    initialize_langchain()
    yield
    # This code would run on shutdown (if needed)
    print("FastAPI app shutting down.")

app = FastAPI(lifespan=lifespan) # Pass the lifespan to the FastAPI app

@app.get("/", response_class=HTMLResponse)
async def read_root():
    # Simple HTML interface for the prototype
    return """
    <!DOCTYPE html>
    <html>
    <head>
        <title>BharatGPT MSME Bot Prototype</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 40px; background-color: #f4f4f4; }
            .container { background-color: #fff; padding: 30px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); max-width: 800px; margin: auto; }
            input[type="text"] { width: calc(100% - 100px); padding: 10px; margin-bottom: 10px; border: 1px solid #ddd; border-radius: 4px; }
            button { padding: 10px 15px; background-color: #007bff; color: white; border: none; border-radius: 4px; cursor: pointer; }
            button:hover { background-color: #0056b3; }
            #chat-output { border: 1px solid #eee; padding: 15px; min-height: 200px; max-height: 400px; overflow-y: auto; background-color: #f9f9f9; border-radius: 4px; margin-top: 20px; }
            .message { margin-bottom: 10px; }
            .user-message { text-align: right; color: #007bff; }
            .bot-message { text-align: left; color: #333; }
            audio { width: 100%; margin-top: 10px; }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>BharatGPT MSME Bot (Prototype)</h1>
            <p>Ask questions related to MSMEs in Hindi (e.g., मुद्रा योजना, पंजीकरण कैसे करें, मार्केटिंग).</p>

            <div id="chat-output"></div>

            <form id="chat-form">
                <input type="text" id="user-input" placeholder="अपना प्रश्न हिंदी में टाइप करें..." autocomplete="off">
                <button type="submit">पूछें</button>
                <button type="button" id="campaign-button">अभियान बनाएं</button>
            </form>

            <h2>अभियान बनाएं (Prototype)</h2>
            <form id="campaign-form">
                <input type="text" id="campaign-product" placeholder="उत्पाद का नाम" required>
                <input type="text" id="campaign-discount" placeholder="छूट प्रतिशत (उदा. 10%)">
                <input type="text" id="campaign-validity" placeholder="वैधता (उदा. 31 जुलाई तक)">
                <button type="submit">अभियान टेक्स्ट जनरेट करें</button>
            </form>
            <div id="campaign-output" style="margin-top: 15px; border: 1px dashed #ccc; padding: 10px; min-height: 50px;"></div>

        </div>

        <script>
            const chatForm = document.getElementById('chat-form');
            const userInput = document.getElementById('user-input');
            const chatOutput = document.getElementById('chat-output');
            const campaignButton = document.getElementById('campaign-button');
            const campaignForm = document.getElementById('campaign-form');
            const campaignProductInput = document.getElementById('campaign-product');
            const campaignDiscountInput = document.getElementById('campaign-discount');
            const campaignValidityInput = document.getElementById('campaign-validity');
            const campaignOutput = document.getElementById('campaign-output');

            async function sendMessage(message) {
                const userMsgDiv = document.createElement('div');
                userMsgDiv.className = 'message user-message';
                userMsgDiv.textContent = `आप: ${message}`;
                chatOutput.appendChild(userMsgDiv);
                userInput.value = '';
                chatOutput.scrollTop = chatOutput.scrollHeight; // Scroll to bottom

                const response = await fetch('/chat', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/x-www-form-urlencoded' },
                    body: `query=${encodeURIComponent(message)}`
                });
                const data = await response.json();

                const botMsgDiv = document.createElement('div');
                botMsgDiv.className = 'message bot-message';
                botMsgDiv.textContent = `बॉट: ${data.response}`;
                chatOutput.appendChild(botMsgDiv);

                if (data.audio_url) {
                    const audio = document.createElement('audio');
                    audio.controls = true;
                    audio.src = data.audio_url;
                    botMsgDiv.appendChild(audio);
                    audio.play(); // Auto-play
                }
                chatOutput.scrollTop = chatOutput.scrollHeight; // Scroll to bottom again
            }

            async function generateCampaign(product, discount, validity) {
                campaignOutput.innerHTML = 'जनरेट कर रहा है...';
                const response = await fetch('/generate_campaign', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/x-www-form-urlencoded' },
                    body: `product=${encodeURIComponent(product)}&discount=${encodeURIComponent(discount)}&validity=${encodeURIComponent(validity)}`
                });
                const data = await response.json();
                campaignOutput.textContent = data.campaign_text;
            }


            chatForm.addEventListener('submit', (e) => {
                e.preventDefault();
                const message = userInput.value.trim();
                if (message) {
                    sendMessage(message);
                }
            });

            campaignForm.addEventListener('submit', (e) => {
                e.preventDefault();
                const product = campaignProductInput.value.trim();
                const discount = campaignDiscountInput.value.trim();
                const validity = campaignValidityInput.value.trim();
                if (product) {
                    generateCampaign(product, discount, validity);
                } else {
                    alert('कृपया उत्पाद का नाम दर्ज करें।');
                }
            });

            window.onload = () => {
                const welcomeDiv = document.createElement('div');
                welcomeDiv.className = 'message bot-message';
                welcomeDiv.textContent = 'बॉट: नमस्कार! मैं भारतजीपीटी एमएसएमई बॉट का प्रोटोटाइप हूँ। मैं आपकी कैसे मदद कर सकता हूँ?';
                chatOutput.appendChild(welcomeDiv);
            };

        </script>
    </body>
    </html>
    """

@app.post("/chat")
async def chat_endpoint(query: str = Form(...)):
    # Main RAG logic
    # initialize_langchain() is called by the lifespan event
    if qa_chain is None: # Fallback check, theoretically should be initialized by lifespan
         return {"response": "बॉट अभी तैयार नहीं है। कृपया कुछ देर प्रतीक्षा करें या LLM मॉडल पाथ जांचें।", "audio_url": None}

    print(f"Received query: {query}")
    try:
        result = qa_chain.invoke({"query": query})
        bot_response = result.get("result", "मुझे इस प्रश्न का उत्तर नहीं मिल रहा है।")
        print(f"Bot response: {bot_response}")

        # Generate TTS audio
        audio_filename = f"audio_{hash(bot_response)}.mp3" # Simple hash for unique name
        audio_path = os.path.join("static", audio_filename)
        # os.makedirs("static", exist_ok=True) # This is already handled in Cell 2
        try:
            tts = gTTS(text=bot_response, lang='hi', slow=False)
            tts.save(audio_path)
            audio_url = f"/static/{audio_filename}"
        except Exception as e:
            print(f"Error generating TTS: {e}")
            audio_url = None

        return {"response": bot_response, "audio_url": audio_url}
    except Exception as e:
        print(f"Error during chat processing: {e}")
        return {"response": "क्षमा करें, कुछ त्रुटि हुई।", "audio_url": None}

@app.post("/generate_campaign")
async def generate_campaign_endpoint(
    product: str = Form(...),
    discount: str = Form(""),
    validity: str = Form("")
):
    # initialize_langchain() is called by the lifespan event
    if llm_model is None: # Fallback check
         return {"campaign_text": "बॉट अभी तैयार नहीं है। कृपया कुछ देर प्रतीक्षा करें या LLM मॉडल पाथ जांचें।"}

    campaign_prompt = f"""एक छोटे व्यवसाय के लिए एक आकर्षक हिंदी मार्केटिंग अभियान टेक्स्ट जनरेट करें।
    उत्पाद/सेवा: {product}
    """
    if discount:
        campaign_prompt += f"छूट: {discount}\n"
    if validity:
        campaign_prompt += f"वैधता: {validity}\n"

    campaign_prompt += """
    अभियान टेक्स्ट:
    """
    try:
        campaign_text = llm_model.invoke(campaign_prompt)
        # Simple cleanup, LLMs can sometimes add extra text or conversational turns
        # This tries to extract the pure campaign text
        if "अभियान टेक्स्ट:" in campaign_text:
            campaign_text = campaign_text.split("अभियान टेक्स्ट:")[-1].strip()
        # Further refine if it includes conversational filler
        if campaign_text.lower().startswith(("यहां एक अभियान टेक्स्ट है:", "यहाँ एक अभियान टेक्स्ट है:", "यह रहा आपका अभियान:", "आपका अभियान टेक्स्ट यहाँ है:", "निश्चित रूप से, यहाँ है:")):
            campaign_text = campaign_text.split(":", 1)[-1].strip()

        return {"campaign_text": campaign_text}
    except Exception as e:
        print(f"Error generating campaign: {e}")
        return {"campaign_text": "अभियान टेक्स्ट जनरेट करने में त्रुटि हुई।"}

# To serve static files (like audio)
from fastapi.staticfiles import StaticFiles
app.mount("/static", StaticFiles(directory="static"), name="static")

In [ ]:
from pyngrok import ngrok
import nest_asyncio
import os

# Apply nest_asyncio for running FastAPI inside a Jupyter environment
nest_asyncio.apply()

# Set your ngrok authtoken from Colab secrets
# Go to the left sidebar in Colab, click the key icon (Secrets), and add a new secret.
# Name: NGROK_AUTH_TOKEN, Value: <Your_Ngrok_Token_Here>
try:
    NGROK_AUTH_TOKEN = os.environ.get("NGROK_AUTH_TOKEN")
    if NGROK_AUTH_TOKEN:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        print("ngrok authtoken set from Colab secrets.")
    else:
        print("WARNING: NGROK_AUTH_TOKEN not found in Colab secrets. ngrok might ask for it or limit usage.")
        print("Please add your ngrok token to Colab secrets for stable operation.")
except Exception as e:
    print(f"Error setting ngrok authtoken: {e}")

# Start ngrok tunnel on port 8000
print("Starting ngrok tunnel...")
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")
print("Open this URL in your browser to access the bot.")

# Run your FastAPI app using uvicorn within the same cell
# This will block the cell, but your FastAPI app will be accessible via the public_url
import uvicorn
print("Starting Uvicorn server...")
uvicorn.run(app, host="0.0.0.0", port=8000)

Please add your ngrok token to Colab secrets for stable operation.
Starting ngrok tunnel...
Public URL: https://725eb37a351c.ngrok-free.app
Open this URL in your browser to access the bot.
Starting Uvicorn server...


INFO:     Started server process [12595]
INFO:     Waiting for application startup.
/tmp/ipython-input-9-2205704039.py:38: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


Initializing LangChain components for the first time...
Loading Embeddings Model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Embeddings Model Loaded.
Loading/Creating ChromaDB at ./chroma_db...


/tmp/ipython-input-9-2205704039.py:44: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


ChromaDB initialized. Documents currently in store: 0
Adding initial Hindi documents to ChromaDB...
Initial documents added.
ChromaDB persisted with initial documents.
Attempting to load LLM from: ./models/OpenHathi-7B-Hi-v0.1-Base-q4_0.gguf


/tmp/ipython-input-9-2205704039.py:65: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist() # Important: Persist the database after adding documents
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


LlamaCpp LLM loaded successfully.
Setting up RetrievalQA Chain...
LangChain components initialized successfully.
INFO:     59.145.191.142:0 - "GET / HTTP/1.1" 200 OK
INFO:     59.145.191.142:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
Received query: मुद्रा योजना का उद्देश्य क्या है?
Bot response: 
---
Mudra Scheme is a scheme launched by the government to provide loans up to ₹10 lakh to small businesses, which helps them grow or start new businesses. यह योजना छोटे और मध्यम आकार के उद्यमों (एस. एम. ई.) को वित्तीय सहायता प्रदान करती है। The objective of this scheme is to create more employment opportunities and contribute to the country's economy.
INFO:     59.145.191.142:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     59.145.191.142:0 - "POST /generate_campaign HTTP/1.1" 200 OK
